# veFVM Distribution

In [1]:
import pandas as pd
import json

vefvm_total = 60000

In [17]:
bsc_mpx_balances = pd.read_csv("../products/bsc-mpx-balances.csv", header=0, names=["address", "bsc_mpx_balance"], dtype={"address": str, "bsc_mpx_balance": float})
bsc_fsmlp_balances = pd.read_csv("../products/bsc-fsmlp-balances.csv", header=0, names=["address", "bsc_fsmlp_balance"], dtype={"address": str, "bsc_fsmlp_balance": float})
bsc_thena_balances = pd.read_csv("../products/bsc-thena-balances.csv", header=0, names=["address", "bsc_thena_lp_balance", "bsc_thena_mpx_balance"], dtype={"address": str, "bsc_thena_lp_balance": float, "bsc_thena_mpx_balance": float})
fantom_equalizer_balances = pd.read_csv("../products/fantom-equalizer-balances.csv", header=0, names=["address", "fantom_equalizer_lp_balance", "fantom_equalizer_mpx_balance"], dtype={"address": str, "fantom_equalizer_lp_balance": float, "fantom_equalizer_mpx_balance": float})
fantom_fsmlp_balances = pd.read_csv("../products/fantom-fsmlp-balances.csv", header=0, names=["address", "fantom_fsmlp_balance"], dtype={"address": str, "fantom_fsmlp_balance": float})
fantom_fvm_balances = pd.read_csv("../products/fantom-fvm-balances.csv", header=0, names=["address", "fantom_fvm_lp_balance", "fantom_fvm_mpx_balance"], dtype={"address": str, "fantom_fvm_lp_balance": float, "fantom_fvm_mpx_balance": float})
fantom_mpx_balances = pd.read_csv("../products/fantom-mpx-balances.csv", header=0, names=["address", "fantom_mpx_balance"], dtype={"address": str, "fantom_mpx_balance": float})
fantom_smpx_balances = pd.read_csv("../products/fantom-smpx-balances.csv", header=0, names=["address", "fantom_smpx_balance"], dtype={"address": str, "fantom_smpx_balance": float})

In [18]:
bsc_combined = pd.merge(bsc_mpx_balances, bsc_thena_balances, on=["address"], how="outer")
bsc_combined = pd.merge(bsc_combined, bsc_fsmlp_balances, on=["address"], how="outer")
bsc_combined.fillna(0, inplace=True)
len(bsc_combined)
bsc_combined.head()

,address,bsc_mpx_balance,bsc_thena_lp_balance,bsc_thena_mpx_balance,bsc_fsmlp_balance
0,0x0B997e226B63FdED87673Bc8B43bA24477b8147A,1.012976e+19,1.332307e+20,5.567542e+21,0.0
1,0x17D3b8142b698bE076eA89339a29FBd36359a734,3.281957e+15,0.000000e+00,0.000000e+00,0.0
2,0x20D5F3f40Dd1783D4e1075fc028732D60D2B9Dd6,1.009509e+17,0.000000e+00,0.000000e+00,0.0
3,0x211A81532DcA439F5f044DDA65f1bA363ccA7A12,8.368108e+15,1.193901e+20,4.989158e+21,0.0
4,0x2728e32174FC95De00695007c9134fA03dC8b434,2.048232e+20,0.000000e+00,0.000000e+00,0.0


In [19]:
ftm_combined = pd.merge(fantom_equalizer_balances, fantom_fsmlp_balances, on=["address"], how="outer")
ftm_combined = pd.merge(ftm_combined, fantom_fvm_balances, on=["address"], how="outer")
ftm_combined = pd.merge(ftm_combined, fantom_mpx_balances, on=["address"], how="outer")
ftm_combined = pd.merge(ftm_combined, fantom_smpx_balances, on=["address"], how="outer")
ftm_combined.fillna(0, inplace=True)
ftm_combined.head()
len(ftm_combined)

2528

In [21]:
combined = pd.merge(bsc_combined, ftm_combined, on=["address"], how="outer")
combined.fillna(0, inplace=True)
len(combined)

2562

In [22]:
combined["total_mpx"] = \
    combined["bsc_mpx_balance"] + \
    combined["bsc_fsmlp_balance"] + \
    combined["bsc_thena_mpx_balance"] + \
    combined["fantom_equalizer_mpx_balance"] + \
    combined["fantom_fvm_mpx_balance"] + \
    combined["fantom_smpx_balance"] + \
    combined["fantom_mpx_balance"]
combined = combined.sort_values(by='address')

In [23]:
# Remove treasury, mpx, etc
with open("../data/excluded.json", "r") as f:
    excluded = json.load(f)
combined = (combined[~combined.address.isin(excluded)])
len(combined)

2561

In [24]:
mpx_total = sum(combined["total_mpx"])
combined['vefvm_balance'] = (combined['total_mpx'] / mpx_total) * vefvm_total * 1e18
sum(combined['vefvm_balance'])

5.999999999999978e+22

In [25]:
combined.to_csv("../products/combined.csv", index=False, float_format='%.0f')

In [26]:
vefvm = combined[["address", "vefvm_balance"]]
vefvm.to_csv("../products/vefvm-balances.csv", index=False, float_format='%.0f')